## 신경망의 엔진: 그래디언트 기반 최적화
* 첫 번째 신경망 예제에 있는 각 층은 입력 데이터를 다음과 같이 변환한다.
* `output = relu(dot(W, input) + b)`
* 텐서 W와 b는 층의 속성처럼 볼 수 있다.
* 가중치(weight) 또는 훈련되는 파라미터(trainable parameter)라고 부른다.
* 초기에는 가중치 행렬이 작은 난수로 채워쳐 있다. (무작위 초기화(random initialization) 단계)
* 피드백 신후에 기초하여 가중치가 점진적으로 조정될 것이다.
* 이런 점진적인 조정 또는 훈련(training)이 머션 러닝 학습의 핵심이다.
* 훈련 반복 루프(training loop)
    1. 훈련 샘플 x와 이에 상응하는 타깃 y의 배치를 추출한다.
    2. x를 사용하여 네트워크를 실행하고 (정방향 패스(forward pass) 단계), 예측 y_pred를 구한다.
    3. y_pred와 y의 차이를 측정하여 이 배치에 대한 네트워크의 손실을 계산한다.
    4. 배치에 대한 손실이 조금 감소되도록 네트워크의 모든 가중치를 업데이트한다.
* 신경망에 사용된 모든 연산이 미분 가능(differentiable)하다는 장점을 사용하여 네트워크 가중치에 대한 손일의 그래디언트(gradient)를 계산
* 그래디언트의 반대 방향으로 가중치를 이동하면 손실이 감소된다.

### 확률적 경사 하강법
* 미분 가능한 함수가 주어지면 이론적으로 이 함수의 최솟값을 해석적으로 구할 수 있다.
* 함수의 최솟값은 변화율이 0인 지점.
* 변화율이 0이 되는 지점을 모두 찾고 이 중에서 어떤 포인트의 함수 값이 가장 작은지 확인하는 것이다.
* 신경망에 적용하면 가장 작은 손실 함수의 값을 만드는 가중치의 조합을 해석적으로 찾는 것을 의미한다.
* `gradient(f)(W) = 0` 을 풀면 해결되는데, N(네트워크의 가중치 개수)개의 변수로 이루어진 다항식.
* N은 종종 수천만 개가 되기 때문에 해석적으로 해결하는 것이 어렵다.
* 미분 가능한 함수를 가지고 있으므로 그래디언트를 계산하여 단계 4를 효울적으로 구현할 수 있다.
* 미니 배치 확률적 경사 하강법 (mini-batch stochastic gradient descent) (미니 배치 SGD)
    1. 훈련 샘플 배치 x와 이에 상응하는 타깃 y를 추출.
    2. x로 네트워크를 실행하고 예측 y_pred를 구한다.
    3. 이 배치에서 y_pred와 y 사이의 오차를 측정하여 네트워크의 손실을 계산한다.
    4. 네트워크의 파라미터에 대한 손실 함수의 그래디언트를 계산한다. (역방향 패스(backward pass)).
    5. 그래디언트의 반대 방향으로 파라미터를 조금 이동시킨다.
* step값을 적절히 고르는 것이 중요하다.
    * 이 값이 너무 작으면 곡선을 따라 내러가는 데 너무 많은 반복이 필요하고 지역 최솟값(local minimum)에 갇힐 수 있다.
    * step이 너무 크면 손실 함수 곡선에서 완전히 임의의 ㅇ위치로 이동시킬 수 있다.
* 미니 배치 SGD 알고리즘의 한 가지 변종은 반복마다 하나의 샘플과 하나의 타깃을 뽑는 것이다.
* 다른 한편으로 극단적인 반대의 경우를 생각해 보면 가용한 모든 데이터를 사용하여 반복을 실행할 수 있다. (배치 SGD(batch SGD))
    * 더 정확하게 업데이트되지만 더 많은 비용이 든다.
* 업데이트할 다음 가중치를 계산할 때 현재 그래디언트 값만 보지 않고 이전에 업데이트된 가중치를 여러 가지 다는 방식으로 고려하는 SGD 변종이 많이 있다.
    * 모멘텀을 사용한 SGD, Adagrad, RMSProp 등이 있다.
* 이련 변종들을 모두 **최적화 방법(optimization method)** 또는 **옵티마이저**
* 여러 변종들에서 사용하는 모멘텀(momentum) 개념은 아주 중요하다.
    * 모멘텀은 SGD에 있는 2개의 문제점인 수렴 속도와 지역 최솟값을 해결한다.
    * 최적화 과정을 손실 각선 위로 작은 공을 굴리는 것으로 생각하면 쉽게 이해할 수 있다.
    * 모멘텀이 충분하면 공이 골짜기에 걷히지 않고 전역 최솟값에 도달할 것이다.
    * 모멘텀은 현재 기울기 값 (현재 가속도)뿐만 아니라 (과거의 가속도로 인한) 현재 속도륻 함께 고려하여 각 단계에서 공을 움직인다.
    * 실전에 적용할 때는 현재 그래디언트 값뿐만 아니라 이전에 업데이터한 파라미터에 기초하여 파라미터 w를 업데이트한다.
    
    ```
    past_velocity = 0.
    momentum = 0.1      # 모멘텀 상수
    while loss > 0.01:  # 최적화 반복 루프
        w, loss, gradient = get_current_parameters()
        velocity = momentum * past_velocity - learning_rate * gradient
        w = w + momentum * velocity - learning_rate * gradient
        past_velocity = velocity
        update_parameter(w)
    ```

### 변화율 연결: 역전파 알고리즘
* 3개의 텐서 연산 a, b, c와 가중치 행렬 W1, W2, W3로 구성된 네트워크 f를 예로 들면
* `f(W1, W2, W3) = a(W1, b(W2, c(W3)))`
* 미적분에서 이렇게 연결된 함수는 연쇄 법칙(chain rule)이라 부르는 다음 항등식 `f(g(x))' = f'(g(x)) * g'(x)`를 사용하여 유도될 수 있다.
* 연쇄 법칙을 신경망의 그래디언트 계산에 적용하여 역전파(Backpropagation) 알고리즘(후진 모드 자동 미분(reverse-mode automatic differentiation)이라고도 부른다.)
* 역전파는 최종 손실 값에서부터 시작한다.
* 손실 값에 각 파라미터가 기여한 정도를 계산하기 위해 연쇄 법칙을 적용하여 최상위 층에서 아휘 층까지 거꾸로 진행된다.